# ![](logo.svg) `doitoml` playground

This is the "behind-the-scenes" view of making an interactive playground: pressing `▶▶` (restart-and-run-all) should reveal a new app panel.

In [ ]:
if __name__ == "__main__" and __import__("platform").machine() == "wasm32":
    %pip install -r requirements.txt

In [ ]:
%reload_ext autoreload
%autoreload 2

This cell imports [dodo.py](./dodo.py), which applies a few patches to make importing `doit` work.

In [ ]:
import dodo

print(dodo.__doc__)

In [ ]:
import os
from pathlib import Path
from tempfile import TemporaryDirectory

import ipylab as L
import ipywidgets as W
import traitlets as T
from doitoml import DoiTOML
from IPython.display import JSON, Markdown

In [ ]:
style = W.HTML(
    """
<style>
.doitoml .jp-widgets-unset-width {
    width: unset !important;
}
.doitoml .jp-OutputArea-prompt {
    display: none
}
.doitoml .jp-OutputArea {
    max-height: 100%;
}
</style>
""",
    layout={"flex": "0"},
)

In [ ]:
if __name__ == "__main__":
    display(style)

In [ ]:
def make_a_file(files: W.Box, name="", text=""):
    ui_name = W.Text(name, placeholder="path to file")
    ui_text = W.Textarea(text, placeholder="file content", layout={"width": "99%"})
    ui_remove = W.Button(
        icon="trash",
        button_style="danger",
        tooltip="remove file",
        _dom_classes=["jp-widgets-unset-width"],
    )
    ui = W.VBox([W.HBox([ui_remove, ui_name]), ui_text])
    ui.add_traits(name=T.Unicode().tag(sync=True), text=T.Unicode().tag(sync=True))
    T.link((ui_name, "value"), (ui, "name"))
    T.link((ui_text, "value"), (ui, "text"))
    ui_name.observe(files.files_changed, "value")
    ui_text.observe(files.files_changed, "value")
    T.dlink((ui_text, "value"), (ui_text, "rows"), lambda x: len(x.splitlines()) + 1)

    @ui_remove.on_click
    def remove(*args):
        files.children = [c for c in files.children if c != ui]

    files.children = [*files.children, ui]

In [ ]:
def count_files(p: Path):
    return len(
        [c for c in p.rglob("*") if not c.is_dir() and "checkpoint" not in str(c)],
    )

In [ ]:
def make_a_file_list():
    files = W.VBox(layout={"display": "block"})
    ui_add_file = W.Button(description="add file", icon="plus", button_style="primary")
    examples = sorted(Path("playground-examples").glob("*"))
    load_label = "load example..."
    options = {load_label: None}
    options.update(
        {
            "{} (files: {})".format(
                p.name.split(".")[0].replace("-", " ").replace("_", "."),
                count_files(p),
            ): p
            for p in examples
            if p.is_dir() and "checkpoint" not in str(p)
        },
    )
    ui_pick = W.Dropdown(description="example", options=options)
    ui_pick.label = load_label

    def on_pick(*_):
        if ui_pick.value is None:
            return
        with ui.hold_sync():
            files.children = []
            for p in sorted(ui_pick.value.rglob("*")):
                if "checkpoint" in str(p):
                    continue
                make_a_file(files, name=p.name, text=p.read_text(encoding="utf-8"))

    ui_pick.observe(on_pick, "value")
    controls = W.HBox([ui_pick, ui_add_file])
    ui = W.VBox([controls, files], layout={"flex": "1"})
    ui.add_traits(files=T.Dict())

    def _files_changed(self, *_):
        ui.files = {f.name: f.text for f in files.children}

    files.files_changed = _files_changed
    files.observe(files.files_changed, "children")

    @ui_add_file.on_click
    def add(*_):
        make_a_file(files)

    return ui

In [ ]:
if __name__ == "__main__":
    file_list = make_a_file_list()
    display(file_list)

In [ ]:
def make_an_output_area(file_list):
    ui_err = W.Output(layout={"flex": "0"})
    ui_mermaid = W.Output(layout={"flex": "1"})
    ui_json = W.Output(layout={"flex": "1"})
    outputs = L.SplitPanel(
        [ui_mermaid, ui_json],
        layout={"flex": "1", "overflow": "hidden"},
    )
    ui = W.VBox([ui_err, outputs], layout={"flex": "1"})
    placeholder = W.HTML("<blockquote>no <code>doitoml</code> yet.</blockquote>")
    with ui_json:
        display(placeholder)
    with ui_mermaid:
        display(placeholder)

    def on_files(*_):
        with TemporaryDirectory() as td:
            tdp = Path(td)
            for name, text in file_list.files.items():
                if not name or not text:
                    continue
                path = tdp / name
                path.write_text(text, encoding="utf-8")
            ui_err.clear_output()
            dt = None
            dt_dict = {}
            ui_err.clear_output()
            ui_json.clear_output()
            ui_mermaid.clear_output()
            with ui_err:
                try:
                    dt = DoiTOML(cwd=td)
                    dt_dict = dt.config.to_dict()
                    dt_dict["env"] = {
                        k: v for k, v in dt_dict["env"].items() if k not in os.environ
                    }
                except Exception as err:
                    display(Markdown(f"**{err.__class__.__name__}**: _{err}_"))
            with ui_json:
                display(JSON(dt_dict, expanded=True) if dt_dict else placeholder)
            with ui_mermaid:
                display(
                    Markdown(dodo.dt2mermaid(dt_dict, "TB"))
                    if dt_dict
                    else placeholder,
                )

    file_list.observe(on_files, "files")
    return ui

In [ ]:
if __name__ == "__main__":
    output_area = make_an_output_area(file_list)
    display(output_area)

In [ ]:
def make_an_app():
    file_list = make_a_file_list()
    outputs = make_an_output_area(file_list)
    main = L.SplitPanel(
        [file_list, outputs],
        orientation="horizontal",
        layout={"flex": "1", "overflow": "hidden"},
    )
    ui_app = W.VBox(
        [main, style],
        layout={"min_height": "400px", "height": "100%", "overflow": "hidden"},
        _dom_classes=["doitoml"],
    )
    return ui_app

In [ ]:
if __name__ == "__main__":
    app = make_an_app()
    display(app)

In [ ]:
def show_an_app_panel(app):
    app = app or make_an_app()
    lab = L.JupyterFrontEnd()
    icon = L.Icon(
        svgstr=Path("logo.svg")
        .read_text(encoding="utf-8")
        .replace("""height="32""", """height="16""")
        .replace("""width="32""", """width="16"""),
    )
    panel = L.Panel([app])
    panel.title.label = "doitoml playground"
    panel.title.icon = icon
    lab.shell.add(panel, "main")

In [ ]:
if __name__ == "__main__":
    show_an_app_panel(app)